# Proyecto Final IH: Recomendador de Codewars|Github

In [1]:
# https://www.codewars.com/users/leaderboard

In [5]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display
import time

from funciones_scraping import get_languages


pd.options.display.max_columns = None

## Fase 1: Obtención de usuarios de forma iterativa

### Leaderboard
Top 500 de codewars

In [7]:
# URL

# url = 'https://www.codewars.com/users/leaderboard'
# html = requests.get(url).content
# soup = BeautifulSoup(html, "lxml")

In [8]:
# usuarios de leaderboard
def get_top_500(url='https://www.codewars.com/users/leaderboard'): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text for e in soup.select('tr a')}


In [9]:
# Salvamos los nombres en un documento

def save_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Save set/list of users to file '''
    with open(filename, 'w') as f: 
        f.write('\n'.join(users)) 
# save_set_users({'hola', 'adios'})

In [10]:
def save_string_users(users, filename='../output/codewars-users.txt'): 
    ''' Save string of users to file '''
    with open(filename, 'w') as f: 
        f.write(users) 
# save_set_users(users)

In [11]:
def add_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Append to file '''
    with open(filename, 'a') as f: 
        f.write(users) 
# add_set_users(users)

In [12]:
def load_set_users(filename='../output/codewars-users.txt'): 
    ''' Read from file '''
    with open(filename, 'r') as f: 
        users = f.readlines()
    return {user.strip() for user in users}
# len(load_set_users())

### Get Social

In [13]:
# get users in clan

# https://www.codewars.com/users/albertogcmr/following
# https://www.codewars.com/users/albertogcmr/followers
# https://www.codewars.com/users/albertogcmr/allies

def get_social(user): 
    res = set()
    for link in ['following', 'followers', 'allies']: 
        url = 'https://www.codewars.com/users/{}/{}'.format(user, link)
        html = requests.get(url).content

        soup = BeautifulSoup(html, "lxml")
        res = res.union({e.text for e in soup.select('table a')})
    return res

# get_social('albertogcmr')

In [14]:
# cálculo de tiempos: 

def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' %  (method.__name__, (te - ts) * 1000) )
        return result

    return timed

@timeit
def print_prueba(a, b):
    print(a, b)
    
print_prueba('a', 'b')

a b
'print_prueba'  0.12 ms


### Iteración para ampliar número de usuarios

A partir de la semilla inicial de 500 obtenemos casi 4000 usuarios de codewars

In [15]:
# top500 = get_top_500()
# total = set().union(top500)
# len(total)

# Cuarta iteración
total = load_set_users('../output/usuarioscodewars-11528.txt')
len(total)

11520

In [ ]:

@timeit
def get_social_from_users_set(users_set): 
    res = users_set
    errors = []
    for i, user in enumerate(users_set): 
        try: 
            res = res.union(get_social(user))
        except: 
            errors.append(user)
            
    save_set_users(users=errors, filename='../errors/social-error.txt')
    return res

total = get_social_from_users_set(total)

# output: total=11528 -> get_social_from_users_set'  11027477.05 ms

In [16]:

len(total)

11520

In [17]:
# Me añado a mí mismo

total = total.union({'albertogcmr', 'boyander'})
for user in get_social('albertogcmr'): 
    total = total.union(get_social(user))
len(total)

11520

### Persistencia de datos en un txt

In [19]:
save_set_users(users=total, filename='../output/usuarioscodewars-iter3.txt')
total = load_set_users('../output/usuarioscodewars-iter3.txt')

In [20]:
len(total)

11520

## Fase 2: Obtención de datos de usuario
### 2.1 API codewars

In [21]:
# GET user

def get_user_api(user='albertogcmr'): 
    ''' Obtenemos el json de la API de codewars '''
    url = 'https://www.codewars.com/api/v1/users/{}'.format(user)
    response = requests.get(url)
    return response.json()

# get_user_api('albertogcmr')

In [22]:
def get_value_from_json(user_json, value='username'): 
    ''' 
    value en nivel 1 del json: 
    possible values = ['username', 'honor', 'clan', 'leaderboardPosition', 'skills']
    '''
    return user_json[value]

def get_score_language(user_json, language): 
    try: 
        score = user_json['ranks']['languages'][language]['score']
    except: 
        score = 0
    return score


def get_scores(user_json): 
    res = {}
    for lang in get_languages(): 
        res[lang] = get_score_language(user_json, lang)
    return res

# get_score_language(user_json=alberto, language='sca')
# get_value_from_json(alberto, 'skills')
# get_scores(alberto)

### 2.2 Web Scrapping codewars

Get stats from user

In [23]:
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

# get_all_stats('albertogcmr')

In [24]:
# def get_languages(url='https://www.codewars.com/kata/latest/my-languages', ignore={'all', 'my languages'}): 
#     html = requests.get(url).content
#     soup = BeautifulSoup(html, "lxml")
#     return {e.text.lower() for e in soup.select('#language_filter option')}.difference(ignore) 

languages = get_languages()
languages

{'bf (beta)',
 'c',
 'c#',
 'c++',
 'clojure',
 'coffeescript',
 'crystal',
 'dart',
 'elixir',
 'elm (beta)',
 'erlang (beta)',
 'f#',
 'fortran (beta)',
 'go',
 'groovy (beta)',
 'haskell',
 'java',
 'javascript',
 'julia (beta)',
 'kotlin (beta)',
 'lua (beta)',
 'nasm (beta)',
 'nim (beta)',
 'objective-c (beta)',
 'ocaml (beta)',
 'php',
 'powershell (beta)',
 'purescript (beta)',
 'python',
 'r (beta)',
 'ruby',
 'rust',
 'scala (beta)',
 'shell',
 'solidity (beta)',
 'sql',
 'swift',
 'typescript'}

In [25]:
def get_row(user): 
    ''' 
    Crea un diccionario con los datos recopilados que se puede
    añadir como fila en nuestro dataframe de pandas
    '''
    user_json = get_user_api(user)
    res = get_all_stats(user)
    res.update(get_scores(user_json))
    return res

def create_row(data, new_user): 
    row = get_row(new_user)
    return data.append(other=row, ignore_index=True)

# ivan = get_row('ijcernicharo')
# alberto = get_row('albertogcmr')

### 2.3 Creación de DataFrame

In [26]:
# Por cada uno de los usuarios recopilados, creamos un registro en df_users
total_n = [u for u in total][:]
len(total_n)

11520

In [27]:
@timeit
def create_df(users): 
    # print(users)
    df_res = pd.DataFrame()
    errors = []
    for i, user in enumerate(users): 
        try: 
            print(i, '\t', user)
            df_res = create_row(data=df_res, new_user=user)
        except: 
            errors.append(user)
    try: 
        save_set_users(users=errors, filename='../errors/df-row-error.txt')
    except: 
        print('Error en la escritura del archivo de error')
        print(errors)
    
    return df_res

In [ ]:
df_users = create_df(total_n)
df_users.head()

# iter 1 'create_df'  9796075.92 ms (500 users)
# iter 2 'create_df'  9796075.92 ms (3939 users)
# iter 3 'create_df'  9796075.92 ms (3939 users)

0 	 brkiesel
1 	 Em-Ant
2 	 Mikeysax
3 	 kimiamania
4 	 Daigle
5 	 bourgui
6 	 Morgane
7 	 parks6
8 	 brevity
9 	 kurbzander
10 	 vivek241
11 	 cesgarbar
12 	 Kokoshinka
13 	 ecasiano
14 	 mjimenez1551
15 	 IllusionOfSkill
16 	 alvalau
17 	 Herceg
18 	 lulof
19 	 wei-dai
20 	 ngcarter
21 	 Lyapunov
22 	 sekulski
23 	 cshutchinson
24 	 deniseyu
25 	 RimantasB
26 	 tomyt
27 	 jasonsims
28 	 ccc111
29 	 ksketo
30 	 tjm2285
31 	 cookfront
32 	 somestud39
33 	 harshsikka123
34 	 tan2
35 	 SpiderBoy
36 	 phyous
37 	 pfatcat
38 	 canCode
39 	 roehlerw
40 	 jmunsch
41 	 wengzilla
42 	 kevinilee
43 	 Mubashir92
44 	 luismiramirez
45 	 halcarleton
46 	 lucaschen321
47 	 irmo322
48 	 Dridge
49 	 GrandLine
50 	 hachanuy
51 	 rachitad123
52 	 VadimPopov
53 	 Halligator
54 	 JamieJamie
55 	 blooo
56 	 Kamyk
57 	 josh_ts
58 	 madden
59 	 nagendra226
60 	 KKOKU
61 	 haozh
62 	 AptFox
63 	 jessynt
64 	 ZebGirouard
65 	 ShadyPenguin
66 	 Lysak
67 	 DQ11
68 	 xiexiaofei
69 	 Jonst
70 	 AskSaikatSinha
71 

526 	 tom-wolters
527 	 Saren-Arterius
528 	 alanrsoares
529 	 g-money
530 	 Juliends
531 	 BarHanSolo
532 	 ekamaks
533 	 mmabraham
534 	 Gloomy_mind
535 	 aelenia
536 	 osman_rasit
537 	 JackSchpeck
538 	 kujofa
539 	 richeee
540 	 Xazzzi
541 	 PaulF
542 	 TITANLUX619
543 	 tzaman
544 	 alKamy
545 	 CraftSpider
546 	 lamagnotti
547 	 rubs33
548 	 seblilj
549 	 ymyyy
550 	 NoobNinja
551 	 loughmiller
552 	 Zarnoch
553 	 Quantm
554 	 black_sun
555 	 Mikhail158
556 	 josestrk
557 	 vgpprasad91
558 	 zhibirc
559 	 PatrickCunanan
560 	 dragonsyde
561 	 ALVO
562 	 saniokOnta
563 	 maximus1984
564 	 mjafri118
565 	 elInfidel
566 	 CameronOMBTS
567 	 metallou
568 	 thedatalife
569 	 jacek-jaskolski
570 	 krikus
571 	 sohanejatin
572 	 moonfly
573 	 BadgerOps
574 	 iamnotaround
575 	 cky
576 	 ArtemSemenov
577 	 ajLapid718
578 	 meitham
579 	 diego.sarlinga
580 	 suraj.dolphy
581 	 Zzzzuev
582 	 whoever112211
583 	 sangyeon.lee
584 	 ahmetkocar
585 	 justingourley
586 	 luohua
587 	 vinodjohn

1044 	 weinee
1045 	 nandrz
1046 	 cuidi
1047 	 Katsuki
1048 	 parabuzzle
1049 	 ggmoy
1050 	 laien
1051 	 lair001
1052 	 qiuhan
1053 	 markpytel
1054 	 Dereje1
1055 	 cstruter
1056 	 wenyang_han
1057 	 ericng414
1058 	 EricSchraf
1059 	 luizoliveiramoreira
1060 	 sak141
1061 	 lizardmill
1062 	 vdhawal
1063 	 codemanki
1064 	 itsthom
1065 	 mckelvincaixa
1066 	 hollyewhite
1067 	 Andriy Nakota
1068 	 hd719
1069 	 ysiso9001
1070 	 cu6upb
1071 	 gardes
1072 	 vladislav322228
1073 	 hgfischer
1074 	 Pshkll
1075 	 GabrielSilva
1076 	 wilson8
1077 	 garcha
1078 	 tuongviatruong
1079 	 darong.mean
1080 	 iwka
1081 	 gannonk08
1082 	 harisbarki
1083 	 amonina1
1084 	 3rogue
1085 	 hkp108
1086 	 jminneman
1087 	 WmBlankenship
1088 	 Marto32
1089 	 rkesler
1090 	 Norboy
1091 	 vsrboth
1092 	 Sokha
1093 	 VolodymyrSlav
1094 	 madathawizard
1095 	 haesoox3
1096 	 crl
1097 	 lawyatt88
1098 	 Joe.
1099 	 cpg93
1100 	 nicoh9
1101 	 Beauny
1102 	 muesli4
1103 	 ialpert
1104 	 nick.strohl
1105 	 wolv

1533 	 stepdar
1534 	 dev26th
1535 	 KacperKorban
1536 	 pppetrov
1537 	 tigarcia
1538 	 LPsyCongroo
1539 	 NathanielTodman
1540 	 sacmac
1541 	 WendyKKelly
1542 	 Dufficus
1543 	 kask
1544 	 jimmybanegas93
1545 	 Apvizzi
1546 	 edokan
1547 	 panagiotisf
1548 	 cheneyshreve
1549 	 nulldevelo
1550 	 v4ld3r5
1551 	 L.Huaso
1552 	 mikescodewarsaccount
1553 	 Andrew Wold
1554 	 krzysztof-grzybek
1555 	 bkaes
1556 	 drew.brokke
1557 	 gomodei
1558 	 JHEo12359
1559 	 jmeshen
1560 	 sohail.shaghasi@gmail.com
1561 	 sri_udara
1562 	 Bikash_grg
1563 	 Ye11ow_Submarin
1564 	 kebien6020
1565 	 courier10pt
1566 	 helloimlaura
1567 	 wrousse
1568 	 jjhuang22
1569 	 tiagoferreira
1570 	 jicksonp
1571 	 CChui
1572 	 trusktr
1573 	 hvk33m
1574 	 hegwest
1575 	 ramoona
1576 	 booz
1577 	 kirillku
1578 	 hida
1579 	 regisbsb
1580 	 massilva
1581 	 sirWill
1582 	 rubeus
1583 	 Mozgosup
1584 	 vholzkamm
1585 	 Jafca
1586 	 Dhammasatva
1587 	 Vitaliiy
1588 	 mactro
1589 	 Maksimka_Blin
1590 	 k2coder
1591 

2016 	 jjtfsalgado
2017 	 Tomeun
2018 	 hentaiLolicon
2019 	 Coquillos
2020 	 fomars
2021 	 Pooh
2022 	 Samson Paul Jnr
2023 	 Elliandir
2024 	 czajkowski
2025 	 livfwd
2026 	 psychonout
2027 	 skhamoud
2028 	 pakiam
2029 	 durfi
2030 	 ChristianHC
2031 	 Ninja737
2032 	 nikitakonan
2033 	 radulov
2034 	 J4CODE
2035 	 ZhangVance
2036 	 hleepage
2037 	 ayubov
2038 	 jimkar02
2039 	 Andriuxa
2040 	 emct
2041 	 FranciscoNogueira
2042 	 oigarash
2043 	 dbyington
2044 	 vunguyen_95
2045 	 darlanmendonca
2046 	 Tak
2047 	 victor-ventum
2048 	 KingAlexJ8
2049 	 sailskisurf23
2050 	 ljy-x
2051 	 dobosi
2052 	 WOnder93
2053 	 dheeraj-br
2054 	 josgilmo
2055 	 antrianis
2056 	 GamZwa
2057 	 ijelonek
2058 	 SeWIRs
2059 	 kjennis
2060 	 Butch_1337
2061 	 weavermedia
2062 	 ito2307
2063 	 xxxxpal
2064 	 alanyam0607
2065 	 nazsatria
2066 	 qyp
2067 	 Prox
2068 	 Praka
2069 	 shreedharshetty
2070 	 SqrTT
2071 	 kristianthrane
2072 	 imatmati
2073 	 pawel13
2074 	 crealee
2075 	 dragonfire
2076 	 maks

...

4929 	 gsakkas
4930 	 Onkel Mo
4931 	 allano3
4932 	 HerrWert
4933 	 nikr
4934 	 FranzSchubert92
4935 	 wesgarrison
4936 	 bamboostgg
4937 	 tuvistavie
4938 	 elderbro
4939 	 colbydauph
4940 	 lukassvec
4941 	 TubalLara
4942 	 chrisburr
4943 	 rfbeiboer
4944 	 Sahib
4945 	 JusticeBoi
4946 	 nikfm
4947 	 musa-karakas
4948 	 yosoul93
4949 	 farhana.jannat
4950 	 NeilSMyers
4951 	 leif
4952 	 lxclxcdsg
4953 	 whst
4954 	 msachi
4955 	 neilzzy
4956 	 kajcia
4957 	 lastikas
4958 	 hizri
4959 	 XavierBoubert
4960 	 luki2000
4961 	 mowithafro
4962 	 azumelzu
4963 	 geneccx
4964 	 Petuna
4965 	 Just David
4966 	 Thomas@zp
4967 	 ovcharik
4968 	 bingecodingMikuya707
4969 	 abeaumont
4970 	 skarthikeyan96
4971 	 NateGlazier
4972 	 miller.a
4973 	 AidaJaspa
4974 	 sana.kumar
4975 	 Carnubak
4976 	 Logan
4977 	 aneurycasado
4978 	 afterlol
4979 	 WilliamShreeve
4980 	 voidm
4981 	 ApplesHaveFeelings
4982 	 Subigya
4983 	 mate-kovacs
4984 	 jcp0578
4985 	 carey_casile
4986 	 d8aninja
4987 	 davidec

5413 	 valeriakori
5414 	 franklinASC5
5415 	 Nachtflug
5416 	 suberg
5417 	 rh1n0man
5418 	 6030999
5419 	 RookieBart
5420 	 trincot
5421 	 D7689
5422 	 ethervoid
5423 	 georgelu
5424 	 kevinthomas89
5425 	 aminoche
5426 	 wisyr
5427 	 EduCode
5428 	 4thana
5429 	 guillaumebreton
5430 	 riverxing
5431 	 SeventhMoment
5432 	 zagovlad
5433 	 sandybeach
5434 	 deepsml
5435 	 GerardEmmen
5436 	 Riky
5437 	 irisapo
5438 	 iamramin
5439 	 Rubyrabbii
5440 	 bryonywatson1
5441 	 spwert
5442 	 ian_phobos
5443 	 andersonar2004
5444 	 TedCart
5445 	 cveneziani
5446 	 karllankford
5447 	 luciomcamargo
5448 	 DcPepper
5449 	 OscarAlvarez
5450 	 Sokovninn
5451 	 nikoala
5452 	 bodisd
5453 	 yorafwj
5454 	 redjalapeno
5455 	 hellooooooo
5456 	 sosimi91
5457 	 maralex
5458 	 laziza93
5459 	 joshvn
5460 	 H-Cox
5461 	 playthefunkymusicwhiteboy
5462 	 ititan
5463 	 shyamrekhawar
5464 	 yjkp92
5465 	 hawkuu
5466 	 NagarjunN
5467 	 lilred
5468 	 annikagarbers
5469 	 sirqiao
5470 	 Loki86
5471 	 sushant12

5902 	 pedro.sampaio
5903 	 TorIronhack
5904 	 qqiangwu
5905 	 Nine and three-quarters
5906 	 hertz
5907 	 coolKID
5908 	 dalinaum
5909 	 erdincmutlu
5910 	 bbrassart
5911 	 DIma716
5912 	 lundibundi
5913 	 luisgerrarrdo
5914 	 hulichao
5915 	 elfryskai
5916 	 hellozombie
5917 	 thomass96
5918 	 tmkelly28
5919 	 Nevix
5920 	 lucymonie
5921 	 4stern
5922 	 ivanosipov
5923 	 tricia.lobo
5924 	 Mosaaleb
5925 	 Kraken01
5926 	 SGMartineau
5927 	 vinodkumar
5928 	 Psldev
5929 	 AppleKatz
5930 	 bgtron
5931 	 JordanGBR
5932 	 tapastro
5933 	 DevEhmeth
5934 	 ANA9963
5935 	 nba-world
5936 	 Selvaticus
5937 	 mmiller_he
5938 	 burce
5939 	 Fisken
5940 	 developenguin
5941 	 krizo
5942 	 chuong
5943 	 kasinator
5944 	 shermpavel
5945 	 MaYue
5946 	 jenm
5947 	 e.sternin
5948 	 codyi
5949 	 g-m-g
5950 	 KataSideKick
5951 	 xadozuk
5952 	 Malorne
5953 	 AndreiAusianik
5954 	 nwilsontech
5955 	 brianmarete
5956 	 isauhn
5957 	 pH77
5958 	 t_t_m
5959 	 Surafelcj
5960 	 ceinstein
5961 	 kossnocorp
5

6387 	 Van Helsing
6388 	 lucy-lucy
6389 	 AlexiaTorres
6390 	 erming
6391 	 Lorberry
6392 	 jjsullivan5196
6393 	 adam-tokarski
6394 	 Asamsig
6395 	 danleavitt0
6396 	 xc-smile
6397 	 leo56
6398 	 lichunlin
6399 	 Tam Borine
6400 	 pre1ude
6401 	 geoffp
6402 	 CaiqueReinhold
6403 	 Narsat
6404 	 MadV
6405 	 Acheh
6406 	 v1z
6407 	 brubru777
6408 	 srane804
6409 	 Mykhailo_Yushchenko
6410 	 hsamuelson
6411 	 Warholm
6412 	 pergk
6413 	 Cric
6414 	 angelanjelly
6415 	 vrushalipatel
6416 	 Karthiga95
6417 	 cold_wind
6418 	 vladimiriacob
6419 	 nishinishi
6420 	 minny
6421 	 Kart17
6422 	 huung
6423 	 bart9h
6424 	 shaleynikov
6425 	 heeeman
6426 	 angham aabed
6427 	 Jmsongz
6428 	 hobovsky
6429 	 cdsar
6430 	 simplosophy
6431 	 djeis97
6432 	 MPanther
6433 	 frhyme
6434 	 tkaghdo
6435 	 deeheber
6436 	 vmetalbr
6437 	 mark-carlson
6438 	 earthday47
6439 	 LimeiMay
6440 	 pratamacool06
6441 	 tienvu95
6442 	 codewarsshoes
6443 	 jmizrachi
6444 	 KhaleelSH
6445 	 eduardosanchezcastillo


6872 	 adamcweitzman
6873 	 Fuji91
6874 	 wasinski
6875 	 iloveYiru
6876 	 nickabbott303
6877 	 christian-mingles
6878 	 narayanswa30663
6879 	 DeRain
6880 	 JCBoorgo
6881 	 godlyfast
6882 	 PaviRaghav
6883 	 WHITE PRIVELEGE
6884 	 yanhackcode15
6885 	 midofit
6886 	 caleblayneheath
6887 	 Syah
6888 	 rrandom
6889 	 AdamVegh
6890 	 qianqian78
6891 	 antonxt
6892 	 Oleg Chubaka
6893 	 chitacan
6894 	 ChristopheRibault
6895 	 danielpclark
6896 	 frenetic_be
6897 	 JDPabst
6898 	 mreinarz
6899 	 Nerhei
6900 	 MrSexyBoy
6901 	 adamxtokyo
6902 	 ZeliCode
6903 	 zavanton123
6904 	 H3llscr3am
6905 	 Mango65
6906 	 showlet
6907 	 satyabratadash89
6908 	 rupali93
6909 	 natalielots
6910 	 aRiveraMerida
6911 	 niklasdorsch
6912 	 guilhermelimak
6913 	 kralos
6914 	 nexticus
6915 	 glebec
6916 	 wuchen
6917 	 vikasm0418
6918 	 daewonyoon
6919 	 andrei-yanovich
6920 	 blab
6921 	 Zmie3
6922 	 gman29
6923 	 SamWS
6924 	 benjamingeiger
6925 	 SherryXueyingLi
6926 	 P-A-Works
6927 	 masho RR
6928 	 i

In [ ]:
# añadimos algunos usuarios más

df_users = create_row(data=df_users, new_user='ijcernicharo')
df_users = create_row(data=df_users, new_user='pablobarrio')
df_users.head()

In [ ]:
df_users.shape

## Fase 3: Persistencia de datos

In [ ]:
df_users.to_csv('../output/df-codewars-iter4.csv')
# df = pd.read_csv('../output/df-codewars-iter4.csv')

In [ ]:
# 'linkedin' in df_users.columns

In [ ]:
df = pd.read_csv('../output/df-codewars-iter4.csv')
df.shape

In [ ]:
df.head()